# Evaluation of a Pipeline and its Components

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial5_Evaluation.ipynb)

To be able to make a statement about the quality of results a question-answering pipeline or any other pipeline in haystack produces, it is important to evaluate it. Furthermore, evaluation allows determining which components of the pipeline can be improved.
The results of the evaluation can be saved as CSV files, which contain all the information to calculate additional metrics later on or inspect individual predictions.

### Prepare environment

#### Colab: Enable the GPU runtime
Make sure you enable the GPU runtime to experience decent speed in this tutorial.
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/master/docs/_src/img/colab_gpu_runtime.jpg">

In [1]:
# Make sure you have a GPU running
!nvidia-smi

Fri Feb 11 12:18:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   61C    P0    53W / 230W |    574MiB /  8116MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

     |████████████████████████████████| 2.1 MB 2.1 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-az0pzg4c/farm-haystack_bff75430e941445491183e7a11cbe632
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-az0pzg4c/farm-haystack_bff75430e941445491183e7a11cbe632
  Resolved https://github.com/deepset-ai/haystack.git to commit 85d309f05ef2f7de226406d1a00684897ed9d319
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 20.0 MB/s eta 0:00:00a 0:00:01
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
     ━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━ 394.5/881.9 MB 33.6 MB/s eta 0:00:15^C
     ━━━━━━━━━━━━━━━━━╺

In [3]:
from haystack.modeling.utils import initialize_device_settings

devices, n_gpu = initialize_device_settings(use_cuda=True)

/home/branden/Code/anaconda3/envs/haystack/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


## Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (eg., in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [ ]:
# If Docker is available: Start Elasticsearch as docker container
# from haystack.utils import launch_es
# launch_es()

# Alternative in Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

chown: changing ownership of 'elasticsearch-7.9.2/modules/x-pack-deprecation/x-pack-deprecation-7.9.2.jar': Operation not permitted
chown: changing ownership of 'elasticsearch-7.9.2/modules/x-pack-deprecation/plugin-descriptor.properties': Operation not permitted
chown: changing ownership of 'elasticsearch-7.9.2/modules/x-pack-deprecation/LICENSE.txt': Operation not permitted
chown: changing ownership of 'elasticsearch-7.9.2/modules/x-pack-deprecation/NOTICE.txt': Operation not permitted
chown: changing ownership of 'elasticsearch-7.9.2/modules/x-pack-deprecation/plugin-security.policy': Operation not permitted
chown: changing ownership of 'elasticsearch-7.9.2/modules/x-pack-deprecation': Operation not permitted
chown: changing ownership of 'elasticsearch-7.9.2/modules/spatial/plugin-descriptor.properties': Operation not permitted
chown: changing ownership of 'elasticsearch-7.9.2/modules/spatial/spatial-7.9.2.jar': Operation not permitted
chown: changing ownership of 'elasticsearch-7.9

SubprocessError: Exception occurred in preexec_fn.

## Fetch, Store And Preprocess the Evaluation Dataset

In [5]:
from haystack.utils import fetch_archive_from_http

# Download evaluation data, which is a subset of Natural Questions development set containing 50 documents with one question per document and multiple annotated answers
doc_dir = "../data/nq"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/nq_dev_subset_v2.json.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

INFO - haystack.utils.import_utils -  Found data stored in `../data/nq`. Delete this first if you really want to fetch new data.


False

In [6]:
# make sure these indices do not collide with existing ones, the indices will be wiped clean before data is inserted
doc_index = "tutorial5_docs"
label_index = "tutorial5_labels"

In [7]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
    host="localhost",
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

In [8]:
from haystack.nodes import PreProcessor

# Add evaluation data to Elasticsearch Document Store
# We first delete the custom tutorial indices to not have duplicate elements
# and also split our documents into shorter passages using the PreProcessor
preprocessor = PreProcessor(
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)

# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.
document_store.add_eval_data(
    filename="../data/nq/nq_dev_subset_v2.json", doc_index=doc_index, label_index=label_index, preprocessor=preprocessor
)

WARNING - haystack.document_stores.utils -  Could not convert an answer for 1 questions.
There were conversion errors for question ids: [7211011040021040393]


## Initialize the Two Components of an ExtractiveQAPipeline: Retriever and Reader

In [9]:
# Initialize Retriever
from haystack.nodes import ElasticsearchRetriever

retriever = ElasticsearchRetriever(document_store=document_store)

# Alternative: Evaluate dense retrievers (DensePassageRetriever or EmbeddingRetriever)
# DensePassageRetriever uses two separate transformer based encoders for query and document.
# In contrast, EmbeddingRetriever uses a single encoder for both.
# Please make sure the "embedding_dim" parameter in the DocumentStore above matches the output dimension of your models!
# Please also take care that the PreProcessor splits your files into chunks that can be completely converted with
#        the max_seq_len limitations of Transformers
# The SentenceTransformer model "all-mpnet-base-v2" generally works well with the EmbeddingRetriever on any kind of English text.
# For more information check out the documentation at: https://www.sbert.net/docs/pretrained_models.html

# from haystack.retriever import DensePassageRetriever, EmbeddingRetriever
# retriever = DensePassageRetriever(document_store=document_store,
#                                   query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#                                   passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#                                   use_gpu=True,
#                                   max_seq_len_passage=256,
#                                   embed_title=True)
# retriever = EmbeddingRetriever(document_store=document_store, model_format="sentence_transformers",
#                                embedding_model="all-mpnet-base-v2")
# document_store.update_embeddings(retriever, index=doc_index)

In [10]:
# Initialize Reader
from haystack.nodes import FARMReader

reader = FARMReader("deepset/roberta-base-squad2", top_k=4, return_no_answer=True)

# Define a pipeline consisting of the initialized retriever and reader
from haystack.pipelines import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# The evaluation also works with any other pipeline.
# For example you could use a DocumentSearchPipeline as an alternative:

# from haystack.pipelines import DocumentSearchPipeline
# pipeline = DocumentSearchPipeline(retriever=retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 7 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0    0    0    0    0    0    0 
INFO - haystack.modeling.infer -  /w\  /w\  /w\  /w\  /w\  /w\  /w\
INFO - haystack.modeling

## Evaluation of an ExtractiveQAPipeline
Here we evaluate retriever and reader in open domain fashion on the full corpus of documents i.e. a document is considered
correctly retrieved if it contains the gold answer string within it. The reader is evaluated based purely on the
predicted answer string, regardless of which document this came from and the position of the extracted span.

The generation of predictions is seperated from the calculation of metrics. This allows you to run the computation-heavy model predictions only once and then iterate flexibly on the metrics or reports you want to generate.


In [11]:
from haystack.schema import EvaluationResult, MultiLabel

# We can load evaluation labels from the document store
eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=False)

## Alternative: Define queries and labels directly

#eval_labels = [
#    MultiLabel(
#        labels=[
#            Label(
#                query="who is written in the book of life",
#                answer=Answer(
#                    answer="every person who is destined for Heaven or the World to Come",
#                    offsets_in_context=[Span(374, 434)]
#                ),
#                document=Document(
#                    id='1b090aec7dbd1af6739c4c80f8995877-0',
#                    content_type="text",
#                    content='Book of Life - wikipedia Book of Life Jump to: navigation, search This article is
#                       about the book mentioned in Christian and Jewish religious teachings...'
#                ),
#                is_correct_answer=True,
#                is_correct_document=True,
#                origin="gold-label"
#            )
#        ]
#    )
#]

# Similar to pipeline.run() we can execute pipeline.eval()
eval_result = pipeline.eval(labels=eval_labels, params={"Retriever": {"top_k": 5}})

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/home/branden/Code/haystack/haystack/modeling/model/prediction_head.py:480: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 24.34 Batches/s]


In [12]:
# The EvaluationResult contains a pandas dataframe for each pipeline node.
# That's why there are two dataframes in the EvaluationResult of an ExtractiveQAPipeline.

retriever_result = eval_result["Retriever"]
retriever_result.head()

,multilabel_id,query,filters,gold_document_contents,content,gold_id_match,answer_match,gold_id_or_answer_match,rank,document_id,gold_document_ids,type,node,eval_mode
0,-7433896329008606045,who is written in the book of life,b'null',"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...","all people considered righteous before God. God has such a book, and to be b...",0.0,0.0,0.0,1.0,fcc5f12d8d1f8f57c34e1a3dc574913f-1,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],document,Retriever,integrated
1,-7433896329008606045,who is written in the book of life,b'null',"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...","tablets as adversaries (of God). Also, according to ib. xxxvi. 10, one who c...",0.0,0.0,0.0,2.0,fcc5f12d8d1f8f57c34e1a3dc574913f-2,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],document,Retriever,integrated
2,-7433896329008606045,who is written in the book of life,b'null',"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",from the citizens' registers. The life which the righteous participate in is...,0.0,0.0,0.0,3.0,fcc5f12d8d1f8f57c34e1a3dc574913f-6,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],document,Retriever,integrated
3,-7433896329008606045,who is written in the book of life,b'null',"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...","the apostles' names are ``written in heaven'' (Luke x. 20), or ``the fellow-...",0.0,0.0,0.0,4.0,fcc5f12d8d1f8f57c34e1a3dc574913f-3,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],document,Retriever,integrated
4,-7433896329008606045,who is written in the book of life,b'null',"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",The Absolutely True Diary of a Part-Time Indian - wikipedia The Absolutely T...,0.0,0.0,0.0,5.0,e9260cbbc129f4246ee8fcfbbe385822-0,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],document,Retriever,integrated


In [13]:
reader_result = eval_result["Reader"]
reader_result.head()

,multilabel_id,query,filters,gold_answers,answer,context,exact_match,f1,rank,document_id,gold_document_ids,offsets_in_document,gold_offsets_in_documents,type,node,eval_mode
0,-7433896329008606045,who is written in the book of life,b'null',[every person who is destined for Heaven or the World to Come],,None,0.0,0.000000,1.0,None,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],"[{'start': 0, 'end': 0}]","[{'start': 374, 'end': 435}]",answer,Reader,integrated
1,-7433896329008606045,who is written in the book of life,b'null',[every person who is destined for Heaven or the World to Come],those whose names are written in the Book of Life from the foundation of the...,"ohn of Patmos. As described, only those whose names are written in the Book ...",0.0,0.083333,2.0,fcc5f12d8d1f8f57c34e1a3dc574913f-3,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],"[{'start': 580, 'end': 662}]","[{'start': 374, 'end': 435}]",answer,Reader,integrated
2,-7433896329008606045,who is written in the book of life,b'null',[every person who is destined for Heaven or the World to Come],only the names of the righteous,. The Psalmist likewise speaks of the Book of Life in which only the names o...,0.0,0.000000,3.0,fcc5f12d8d1f8f57c34e1a3dc574913f-1,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],"[{'start': 502, 'end': 533}]","[{'start': 374, 'end': 435}]",answer,Reader,integrated
3,-7433896329008606045,who is written in the book of life,b'null',[every person who is destined for Heaven or the World to Come],"Arnold Spirit Jr.,",person narrative from the perspective of Native American teenager Arnold Spi...,0.0,0.000000,4.0,e9260cbbc129f4246ee8fcfbbe385822-0,[fcc5f12d8d1f8f57c34e1a3dc574913f-0],"[{'start': 1120, 'end': 1138}]","[{'start': 374, 'end': 435}]",answer,Reader,integrated
0,-8282260053644253827,who was the girl in the video brenda got a baby,b'null',[Ethel ``Edy'' Proctor],her cousin,ng a story in the newspaper of a 12-year-old girl getting pregnant by her co...,0.0,0.000000,1.0,965a125f65658579529b39f8e4344969-3,[965a125f65658579529b39f8e4344969-3],"[{'start': 423, 'end': 433}]","[{'start': 181, 'end': 202}]",answer,Reader,integrated


In [14]:
# We can filter for all documents retrieved for a given query
retriever_book_of_life = retriever_result[retriever_result["query"] == "who is written in the book of life"]

In [15]:
# We can also filter for all answers predicted for a given query
reader_book_of_life = reader_result[reader_result["query"] == "who is written in the book of life"]

In [16]:
# Save the evaluation result so that we can reload it later and calculate evaluation metrics without running the pipeline again.
eval_result.save("../")

## Calculating Evaluation Metrics
Load an EvaluationResult to quickly calculate standard evaluation metrics for all predictions,
such as F1-score of each individual prediction of the Reader node or recall of the retriever.
To learn more about the metrics, see [Evaluation Metrics](https://haystack.deepset.ai/guides/evaluation#metrics-retrieval)

In [17]:
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

Retriever - Recall (single relevant document): 0.38
Retriever - Recall (multiple relevant documents): 0.38
Retriever - Mean Reciprocal Rank: 0.23433333333333334
Retriever - Precision: 0.07600000000000001
Retriever - Mean Average Precision: 0.23433333333333334
Reader - F1-Score: 0.7554795204795204
Reader - Exact Match: 0.72


## Generating an Evaluation Report
A summary of the evaluation results can be printed to get a quick overview. It includes some aggregated metrics and also shows a few wrongly predicted examples.

In [18]:
pipeline.print_eval_report(saved_eval_result)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit:  0.42
                        | recall_single_hit_top_1:  0.42
                        |
                      Reader
                        |
                        | exact_match:  0.72
                        | exact_match_top_1:  0.48
                        | f1: 0.755
                        | f1_top_1:   0.5
                        |
                      Output

                Wrong Retriever Examples
Query: 
 	who is written in the book of life
Gold Document Ids: 
 	fcc5f12d8d1f8f57c34e1a3dc574913f-0
Metrics: 
 	recall_multi_hit: 0.0
 	recall_single_hit: 0.0
 	precision: 0.0
 	map: 0.0
 	mrr: 0.0
 	ndcg: 0.0
Documents: 
 	content: all people considered righteous before God. God has such a book, and to be

## Advanced Evaluation Metrics
As an advanced evaluation metric, semantic answer similarity (SAS) can be calculated. This metric takes into account whether the meaning of a predicted answer is similar to the annotated gold answer rather than just doing string comparison.
To this end SAS relies on pre-trained models. For English, we recommend "cross-encoder/stsb-roberta-large", whereas for German we recommend "deepset/gbert-large-sts". A good multilingual model is "sentence-transformers/paraphrase-multilingual-mpnet-base-v2".
More info on this metric can be found in our [paper](https://arxiv.org/abs/2108.06130) or in our [blog post](https://www.deepset.ai/blog/semantic-answer-similarity-to-evaluate-qa).

In [ ]:
advanced_eval_result = pipeline.eval(
    labels=eval_labels, params={"Retriever": {"top_k": 1}}, sas_model_name_or_path="cross-encoder/stsb-roberta-large"
)

metrics = advanced_eval_result.calculate_metrics()
print(metrics["Reader"]["sas"])

## Isolated Evaluation Mode to Understand Upper Bounds of the Reader's Performance
The isolated node evaluation uses labels as input to the reader node instead of the output of the preceeding retriever node.
Thereby, we can additionally calculate the upper bounds of the evaluation metrics of the reader.

In [24]:
eval_result_with_upper_bounds = pipeline.eval(
    labels=eval_labels, params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 5}}, add_isolated_node_eval=True
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 28.51 Batches/s]


In [25]:
pipeline.print_eval_report(eval_result_with_upper_bounds)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit:  0.42
                        | recall_single_hit_top_1:  0.42
                        |
                      Reader
                        |
                        | exact_match upper bound:  0.72
                        | exact_match:  0.72
                        | exact_match_top_1:  0.48
                        | f1 upper bound: 0.798
                        | f1: 0.758
                        | f1_top_1:   0.5
                        |
                      Output

                Wrong Retriever Examples
Query: 
 	who is written in the book of life
Gold Document Ids: 
 	fcc5f12d8d1f8f57c34e1a3dc574913f-0
Metrics: 
 	recall_multi_hit: 0.0
 	recall_single_hit: 0.0
 	precision: 0.0
 	map: 0.0
 	mrr: 0.0
 	nd

## Evaluation of Individual Components: Retriever
Sometimes you might want to evaluate individual components, for example, if you don't have a pipeline but only a retriever or a reader with a model that you trained yourself.
Here we evaluate only the retriever, based on whether the gold_label document is retrieved.

In [21]:
## Evaluate Retriever on its own
retriever_eval_results = retriever.eval(top_k=5, label_index=label_index, doc_index=doc_index)
# Retriever Recall is the proportion of questions for which the correct document containing the answer is
# among the correct documents
print("Retriever Recall:", retriever_eval_results["recall"])
# Retriever Mean Avg Precision rewards retrievers that give relevant documents a higher rank
print("Retriever Mean Avg Precision:", retriever_eval_results["map"])

INFO - haystack.nodes.retriever.base -  Performing eval queries...
100%|██████████| 25/25 [00:00<00:00, 139.48it/s]
INFO - haystack.nodes.retriever.base -  For 19 out of 25 questions (76.00%), the answer was in the top-5 candidate passages selected by the retriever.


Retriever Recall: 0.76
Retriever Mean Avg Precision: 0.4686666666666667


## Evaluation of Individual Components: Reader
Here we evaluate only the reader in a closed domain fashion i.e. the reader is given one query
and its corresponding relevant document and metrics are calculated on whether the right position in this text is selected by
the model as the answer span (i.e. SQuAD style)

In [22]:
# Evaluate Reader on its own
reader_eval_results = reader.eval(
    document_store=document_store, device=devices[0], label_index=label_index, doc_index=doc_index
)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
# reader_eval_results = reader.eval_on_file("../data/nq", "nq_dev_subset_v2.json", device=device)

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
# Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
# Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

INFO - haystack.nodes.reader.farm -  Performing Evaluation using top_k_per_candidate = 3 
and consequently, QuestionAnsweringPredictionHead.n_best = 4. 
This deviates from FARM's default where QuestionAnsweringPredictionHead.n_best = 5
Evaluating:   0%|          | 0/33 [00:00<?, ?it/s]/home/branden/Code/haystack/haystack/modeling/model/prediction_head.py:480: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Evaluating: 100%|██████████| 33/33 [00:21<00:00,  1.56it/s]

Reader Top-N-Accuracy: 99.09208819714657
Reader Exact Match: 84.95460440985732
Reader F1-Score: 85.48643376852016


## About us

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.  
  
Some of our other work: 
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)
- [FARM](https://github.com/deepset-ai/FARM)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Slack](https://haystack.deepset.ai/community/join) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://www.deepset.ai/jobs)